In [3]:
import os
from pathlib import Path
import pandas as pd

# Load data - fuel_train includes fuel consumption in segments
DATA_ORIGIN = Path("data")
DATA_TRAINING = os.path.join(DATA_ORIGIN, "flights_train")
DATA_SUB = os.path.join(DATA_ORIGIN, "flights_rank")
TARGET_PATH = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns")
TARGET_PATH2 = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_2")

TARGET_PATH_SUB = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_sub")
TARGET_PATH_SUB2 = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_sub_2")


# paths
source_dir = Path(DATA_TRAINING)
target_dir = Path(TARGET_PATH)
target_dir.mkdir(parents=True, exist_ok=True)


In [17]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyarrow import flight

import os
from pathlib import Path
import pandas as pd

# Load data - fuel_train includes fuel consumption in segments
DATA_ORIGIN = Path("data")
DATA_TRAINING = os.path.join(DATA_ORIGIN, "flights_train")
DATA_SUB = os.path.join(DATA_ORIGIN, "flights_rank")
TARGET_PATH = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns")
TARGET_PATH2 = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_2")
TARGET_TRAIN_CLEAN = os.path.join(DATA_ORIGIN, "flights_cleaned_train")

TARGET_PATH_SUB = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_sub")
TARGET_PATH_SUB2 = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_sub_2")
TARGET_SUB_CLEAN = os.path.join(DATA_ORIGIN, "flights_cleaned_rank")
TARGET_SUB_CLEAN = os.path.join(DATA_ORIGIN, "flights_test_rank")


DATA_FINAL = os.path.join(DATA_ORIGIN, "flights_final")
TARGET_FINAL_CLEAN = os.path.join(DATA_ORIGIN, "flights_test_final")

TARGET_TRAIN_CLEAN = os.path.join(DATA_ORIGIN, "flights_test_train")


# paths
source_dir = Path(DATA_TRAINING)
target_dir = Path(TARGET_PATH)
target_dir.mkdir(parents=True, exist_ok=True)

import polars as pl
import os

# to do, first delete if altitude changing over time is to big most of the time only one entries altitude change in one minute max 60 m pro seconds

# groundspeed change also if its in cruise high altitudes above a certain value or if the steps here are also to high or low, make null and interpolate +20000 altitude and lower then 110 GS and also check for minute segments the delta,
# also check within 1min groundspeed delta to be within 55
# vertical rate over 8500 delete or handle and also if in cruise more than 2000
# Keep data format int to int
# function to search in other dataset the specific flight id and time
FLIGHTS_DATA = os.path.join(DATA_ORIGIN, "flights_test_train")
SUB_FLIGHTS_DATA = os.path.join(DATA_ORIGIN, "flights_test_rank")
import polars as pl


def flight_search(flight, start, end, data="TRAIN"):
    """
    Returns a segment based on [start, end] for a flight:
      - picks the nearest timestamps to start/end
      - returns rows from start_idx to end_idx
      - adds deviation info (seconds) and the nearest start/end actually used
    """
    # get the flight for the fuel column
    path = ""
    if data == "TRAIN":
        path = os.path.join(FLIGHTS_DATA, f"{flight}.parquet")
    elif data == "RANK":
        path = os.path.join(SUB_FLIGHTS_DATA, f"{flight}.parquet")

    if not os.path.exists(path):
        print(f"[WARN] missing file: {path}")
        return pl.DataFrame()

    df_flight = pl.read_parquet(path)

    # search exact time if not the closest one
    ts = df_flight.get_column("timestamp")

    # nearest indices (work in ms to avoid unit issues)
    start_idx = int(((ts - start).dt.cast_time_unit("ms").abs()).arg_min())
    end_idx = int(((ts - end).dt.cast_time_unit("ms").abs()).arg_min())

    start_ts = ts.item(start_idx)
    end_ts = ts.item(end_idx)

    # signed deviations in seconds (nearest - requested)
    start_dev_s = abs((start_ts - start)).total_seconds()
    end_dev_s = abs((end_ts - end).total_seconds())
    df_flight = df_flight.sort("timestamp")

    threshold_gs = 40
    threshold_alt = 60

    df_diff = df_flight.with_columns(
        pl.col("timestamp").diff().abs().alias("timestamp_diff_abs"),
        pl.col("latitude").diff().abs().alias("lat_diff_abs"),
        pl.col("longitude").diff().abs().alias("lon_diff_abs"),
        pl.col("altitude").diff().abs().alias("altitude_abs"),
        pl.col("groundspeed").diff().abs().alias("gs_diff_abs"),
        pl.col("track").diff().abs().alias("track_abs"),
        pl.col("vertical_rate").diff().abs().alias("vertical_rate_abs"),
    )

    df_cleaned = df_diff.with_columns(
        pl.when(pl.col("gs_diff_abs") > threshold_gs)
        .then(None)  # or pl.lit(None)
        .otherwise(pl.col("groundspeed"))
        .alias("groundspeed_cleaned")
    )
    df_cleaned = df_diff.with_columns(
        pl.when(pl.col("altitude_abs") / pl.col("timestamp_diff_abs") > threshold_alt)
        .then(None)  # or pl.lit(None)
        .otherwise(pl.col("groundspeed"))
        .alias("groundspeed_cleaned")
    )

    if start_dev_s or end_dev_s > 60000:
        segment = False
    else:
        segment = df_flight.filter(
            (pl.col("timestamp") >= start_ts) & (pl.col("timestamp") <= end_ts)
        ).sort("timestamp")

        segment = segment.with_columns(pl.lit(start_dev_s).alias("dev_start"),
                                       pl.lit(end_dev_s).alias("dev_end"))
    return segment


import os
import polars as pl
from pathlib import Path

cols_interpolate = ["longitude", "latitude", "altitude", "groundspeed", "track", "vertical_rate"]


def clean_data(df_flight):
    df_flight = df_flight.sort("timestamp")

    # thresholds
    threshold_gs_low, threshold_gs_alt = 200, 20000  # low GS at high altitude
    threshold_gs_rate = 2.0  # kt per second
    threshold_gs_up = 700  # hard upper cap in kt
    threshold_alt = 60.0  # ft per second

    # diffs
    df_diff = df_flight.with_columns(
        (
                pl.col("timestamp")
                .diff()
                .cast(pl.Int64) / 1_000_000_000.0  # -> seconds
        ).alias("dt_s"),
        pl.col("latitude").diff().abs().alias("lat_diff_abs"),
        pl.col("longitude").diff().abs().alias("lon_diff_abs"),
        pl.col("altitude").diff().abs().alias("altitude_abs"),
        pl.col("groundspeed").diff().abs().alias("gs_diff_abs"),
        pl.col("track").diff().abs().alias("track_abs"),
        pl.col("vertical_rate").diff().abs().alias("vertical_rate_abs"),
    )

    df_file = df_diff

    # 1) low groundspeed at high altitude -> implausible
    df_file = df_file.with_columns(
        pl.when(
            (pl.col("groundspeed") < threshold_gs_low)
            & (pl.col("altitude") > threshold_gs_alt)
        )
        .then(None)
        .otherwise(pl.col("groundspeed"))
        .alias("groundspeed")
    )

    # 2) too fast change in GS: |ΔGS| / dt_s > threshold_gs_rate
    df_file = df_file.with_columns(
        pl.when(
            pl.col("dt_s").is_not_null()
            & (pl.col("dt_s") > 0)
            & ((pl.col("gs_diff_abs") / pl.col("dt_s")) > threshold_gs_rate)
        )
        .then(None)
        .otherwise(pl.col("groundspeed"))
        .alias("groundspeed")
    )

    # 3) hard upper cap on GS
    df_file = df_file.with_columns(
        pl.when(pl.col("groundspeed") > threshold_gs_up)
        .then(None)
        .otherwise(pl.col("groundspeed"))
        .alias("groundspeed")
    )

    # 4) remove impossible altitude jumps
    df_file = df_file.with_columns(
        pl.when(
            pl.col("dt_s").is_not_null()
            & (pl.col("dt_s") > 0)
            & ((pl.col("altitude_abs") / pl.col("dt_s")) > threshold_alt)
        )
        .then(None)
        .otherwise(pl.col("altitude"))
        .alias("altitude")
    )

    # interpolate in pandas
    pdf = df_file.to_pandas().sort_values("timestamp").set_index("timestamp")

    pdf[cols_interpolate] = pdf[cols_interpolate].interpolate(method="time")

    # round GS, keep nullable integer
    pdf["groundspeed"] = pdf["groundspeed"].round().astype("Int64")

    pdf = pdf.reset_index()
    df_out = pl.from_pandas(pdf)
    return df_out


# driver
do_clean = True
source_dir = Path(DATA_TRAINING)
target_dir = Path(TARGET_TRAIN_CLEAN)
target_dir.mkdir(parents=True, exist_ok=True)

if do_clean:
    for file in source_dir.glob("*.parquet"):
        print(f"Processing {file.name} ...")
        df = pl.read_parquet(file)
        df_clean = clean_data(df)
        out_path = target_dir / file.name  # same filename
        df_clean.write_parquet(out_path)

Processing prc770822360.parquet ...
Processing prc770831136.parquet ...
Processing prc770835414.parquet ...
Processing prc770844923.parquet ...
Processing prc770847190.parquet ...
Processing prc770852254.parquet ...
Processing prc770853645.parquet ...
Processing prc770859206.parquet ...
Processing prc770860078.parquet ...
Processing prc770864956.parquet ...
Processing prc770866486.parquet ...
Processing prc770867379.parquet ...
Processing prc770868424.parquet ...
Processing prc770870642.parquet ...
Processing prc770872129.parquet ...
Processing prc770875579.parquet ...
Processing prc770876146.parquet ...
Processing prc770876159.parquet ...
Processing prc770876744.parquet ...
Processing prc770878608.parquet ...
Processing prc770879349.parquet ...
Processing prc770881363.parquet ...
Processing prc770882585.parquet ...
Processing prc770882767.parquet ...
Processing prc770885136.parquet ...
Processing prc770887177.parquet ...
Processing prc770887278.parquet ...
Processing prc770887555.parq

In [27]:
# function to search in other dataset the specific flight id and time
source_dir = Path(DATA_SUB)
target_dir = Path(TARGET_PATH_SUB2)
FLIGHTS_RAW = os.path.join(DATA_ORIGIN, "flights_train")
FLIGHTS_CLEAN = os.path.join(DATA_ORIGIN, "flights_test_train")

FLIGHTS_DATA = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns")
FLIGHTS_DATA2 = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_2")

SUB_FLIGHTS_RAW = os.path.join(DATA_ORIGIN, "flights_rank")
# SUB_FLIGHTS_CLEAN = os.path.join(DATA_ORIGIN, "flights_cleaned_rank_v2")
SUB_FLIGHTS_CLEAN = os.path.join(DATA_ORIGIN, "flights_test_rank")




SUB_FLIGHTS_DATA = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_sub")
SUB_FLIGHTS_DATA2 = os.path.join(DATA_ORIGIN, "postprocessed", "interpolated_all_columns_sub_2")

FINAL_RAW = os.path.join(DATA_ORIGIN, "flights_final")
FINAL_CLEAN = os.path.join(DATA_ORIGIN, "flights_test_final")

import polars as pl

DEVIATION_LIMIT_S = 60000  # ~100 minutes

def flight_search(flight, start, end, data="TRAIN") -> pl.DataFrame:
    """
    Returns rows between the nearest timestamps to [start, end] (inclusive).
    Skips if either edge deviates > DEVIATION_LIMIT_S seconds.
    Handles reversed windows and ensures timestamp dtype/sort.
    """
    # ---- 1) resolve path based on `data` flag ----
    if data == "TRAIN":
        path = os.path.join(FLIGHTS_CLEAN, f"{flight}.parquet")
    elif data == "FINAL":
        path_primary = os.path.join(FINAL_CLEAN, f"{flight}.parquet")
        if os.path.exists(path_primary):
            path = path_primary
        else:
            path_fallback = os.path.join(SUB_FLIGHTS_CLEAN, f"{flight}.parquet")
            if os.path.exists(path_fallback):
                path = path_fallback
            else:
                print(
                    f"[WARN] missing file for flight {flight}: "
                    f"{path_primary} and {path_fallback}"
                )
                return pl.DataFrame()
    else:  # RANK / other
        path = os.path.join(SUB_FLIGHTS_CLEAN, f"{flight}.parquet")

    if not os.path.exists(path):
        print(f"[WARN] missing file: {path}")
        return pl.DataFrame()

    df_flight = pl.read_parquet(path)

    # Ensure timestamp dtype & sort
    if df_flight.schema.get("timestamp") != pl.Datetime:
        df_flight = df_flight.with_columns(
            pl.col("timestamp").str.strptime(pl.Datetime, strict=False)
        )
    df_flight = df_flight.sort("timestamp")

    if df_flight.is_empty():
        return pl.DataFrame()

    ts = df_flight.get_column("timestamp")

    # nearest indices (work in ms to avoid unit issues)
    start_idx = int(((ts - start).dt.cast_time_unit("ms").abs()).arg_min())
    end_idx   = int(((ts - end).dt.cast_time_unit("ms").abs()).arg_min())

    start_ts = ts.item(start_idx)
    end_ts   = ts.item(end_idx)

    # keep chronological order
    if end_ts < start_ts:
        start_ts, end_ts = end_ts, start_ts
        start_idx, end_idx = end_idx, start_idx

    # deviations in seconds (absolute)
    start_dev_s = abs((start_ts - start).total_seconds())
    end_dev_s   = abs((end_ts - end).total_seconds())

    # correct thresholding
    if (start_dev_s > DEVIATION_LIMIT_S) or (end_dev_s > DEVIATION_LIMIT_S):
        return pl.DataFrame()

    segment = df_flight.filter(
        (pl.col("timestamp") >= start_ts) & (pl.col("timestamp") <= end_ts)
    ).sort("timestamp")

    if segment.is_empty():
        return pl.DataFrame()

    return segment.with_columns(
        pl.lit(start_dev_s).alias("dev_start"),
        pl.lit(end_dev_s).alias("dev_end")
    )


def flight_search_extra(flight: str, start, end, context: int = 5, data="TRAIN") -> pl.DataFrame:
    """
    Return a segment AROUND [start,end] with ±context rows, plus metadata.
    If nearest start/end deviate > DEVIATION_LIMIT_S, return an empty DataFrame.
    """
    path = ""
    if data == "TRAIN":
        path = os.path.join(FLIGHTS_DATA, f"{flight}.parquet")
    elif data == "RANK":
        path = os.path.join(SUB_FLIGHTS_DATA, f"{flight}.parquet")

    if not os.path.exists(path):
        print(f"[WARN] missing file: {path}")
        return pl.DataFrame()

    df_flight = pl.read_parquet(path)

    # Ensure timestamp dtype & sort
    if df_flight.schema.get("timestamp") != pl.Datetime:
        df_flight = df_flight.with_columns(
            pl.col("timestamp").str.strptime(pl.Datetime, strict=False)
        )
    df_flight = df_flight.sort("timestamp")

    if df_flight.is_empty():
        return pl.DataFrame()

    ts = df_flight.get_column("timestamp")

    # nearest indices
    start_idx = int(((ts - start).dt.cast_time_unit("ms").abs()).arg_min())
    end_idx   = int(((ts - end).dt.cast_time_unit("ms").abs()).arg_min())

    start_ts = ts.item(start_idx)
    end_ts   = ts.item(end_idx)

    # keep chronological order
    if end_ts < start_ts:
        start_ts, end_ts = end_ts, start_ts
        start_idx, end_idx = end_idx, start_idx

    # deviations (absolute seconds)
    dev_start_s = abs((start_ts - start).total_seconds())
    dev_end_s   = abs((end_ts - end).total_seconds())

    # hard skip on big deviation
    if (dev_start_s > DEVIATION_LIMIT_S) or (dev_end_s > DEVIATION_LIMIT_S):
        pass
        #return pl.DataFrame()

    # window with context
    i0 = max(start_idx - context, 0)
    i1 = min(end_idx + context, df_flight.height - 1)
    length = i1 - i0 + 1

    segment = df_flight.slice(i0, length).with_columns(
        pl.lit(flight).alias("flight_id"),
        pl.lit(start).alias("req_start"),
        pl.lit(end).alias("req_end"),
        pl.lit(start_ts).alias("nearest_start"),
        pl.lit(end_ts).alias("nearest_end"),
        pl.lit(dev_start_s).alias("dev_start_s"),
        pl.lit(dev_end_s).alias("dev_end_s"),
        (pl.col("timestamp").is_between(start_ts, end_ts, closed="both")).alias("in_window"),
        pl.when(pl.col("timestamp").is_between(start_ts, end_ts, closed="both"))
          .then(pl.lit("yes")).otherwise(pl.lit("no")).alias("in_window_str"),
    )

    return segment

# combine dataframes


def dataframes_combine(df_fuel: pl.DataFrame, df_flight: pl.DataFrame) -> pl.DataFrame:
    joined = df_flight.join(
        df_fuel.select([
            "flight_id",
            pl.col("start").alias("segment_start"),
            pl.col("end").alias("segment_end"),
            "fuel_kg",  # add other flight cols you want
            # e.g. "aircraft_type", "origin_icao", "destination_icao"
        ]),
        on="flight_id",
        how="inner",
    )
    return joined


# check if number NULLS is more than normal entries for lat, long, alt

def needs_more_context(df: pl.DataFrame, min_rows: int = 15) -> bool:
    """Return True if we have < min_rows rows where latitude, longitude, altitude are all non-null."""
    if df.is_empty():
        return True
    non_null_rows = df.select(REQ_COLS).drop_nulls().height
    return non_null_rows < min_rows

# create final dataset
# choose which raw columns to keep from the segment
KEEP_COLS = [
    "flight_id", "timestamp", "start", "end",
     "fuel_kg", "typecode",
    "latitude", "longitude", "altitude",          # adjust names to yours
    "groundspeed", "track", "vertical_rate", "source"    # optional
]

# signals to summarize (median + delta=max-min)
STAT_COLS = [
    "latitude", "longitude", "altitude",
    "groundspeed", "track", "vertical_rate",
    "mach", "tas", "cas",   # newly added
]

def build_final_row(segment: pl.DataFrame) -> pl.DataFrame:
    if segment.is_empty():
        return pl.DataFrame()

    # ensure chronological order so "first"/"last" are by time
    if "timestamp" in segment.columns:
        segment = segment.sort("timestamp")

    have = set(segment.columns)
    aggs: list[pl.Expr] = []

    # -------- metadata --------
    if "flight_id" in have:
        aggs.append(pl.col("flight_id").first().alias("flight_id"))
    if "typecode" in have:
        aggs.append(pl.col("typecode").first().alias("typecode"))
    if "source" in have:
        aggs.append(pl.col("source").first().alias("source"))
    if "fuel_kg" in have:
        aggs.append(pl.col("fuel_kg").first().alias("fuel_kg"))
    if "nearest_start" in have:
        aggs.append(pl.col("nearest_start").first().alias("nearest_start"))
    if "nearest_end" in have:
        aggs.append(pl.col("nearest_end").first().alias("nearest_end"))
    if "dev_start" in have:
        aggs.append(pl.col("dev_start").first().alias("dev_start_s"))
    if "dev_end" in have:
        aggs.append(pl.col("dev_end").first().alias("dev_end_s"))
    if "wake" in have:
        aggs.append(pl.col("wake").first().alias("wake"))
    if "engine_type" in have:
        aggs.append(pl.col("engine_type").first().alias("engine_type"))
    if "engine_number" in have:
        aggs.append(pl.col("engine_number").first().alias("engine_number"))

    # flight_duration in source is minutes → convert to seconds
    if "flight_duration" in have:
        aggs.append(
            (pl.col("flight_duration").first() * 60.0).alias("total_flight_duration_s")
        )

    # -------- full flight start/end (takeoff / landing) --------
    # here start/end are the full-flight timestamps
    if {"start", "end"}.issubset(have):
        aggs += [
            pl.col("start").min().alias("takeoff"),
            pl.col("end").max().alias("landing"),
        ]

    # -------- segment duration (segment_start → segment_end, in seconds) --------
    if {"segment_start", "segment_end"}.issubset(have):
        aggs.append(
            (
                (
                    pl.col("segment_end").cast(pl.Int64).max()
                    - pl.col("segment_start").cast(pl.Int64).min()
                )
                / 1_000_000          # Datetime[µs] → seconds; adjust if dtype differs
            ).alias("segment_duration_s")     # segment duration
        )

    # -------- segment start position in flight (seconds + percentage) --------
    # segment_start_time_s = segment_start - takeoff
    if {"segment_start", "start", "flight_duration"}.issubset(have):
        seg_start_s = (
            (
                pl.col("segment_start").cast(pl.Int64).min()
                - pl.col("start").cast(pl.Int64).min()
            )
            / 1_000_000      # Datetime[µs] → seconds
        )

        aggs.append(seg_start_s.alias("segment_start_time_s"))
        aggs.append(
            (
                seg_start_s /
                (pl.col("flight_duration").first() * 60.0)  # minutes → seconds
            ).alias("segment_start_percentage")
        )

    # -------- stats over numeric flight variables --------
    for c in STAT_COLS:
        if c in have:
            aggs += [
                pl.col(c).mean().alias(f"{c}_mean"),
                (pl.col(c).last() - pl.col(c).first()).alias(f"{c}_delta"),
            ]

    # -------- phase percentages within the segment --------
    if "flightphase" in have:
        aggs += [
            (pl.col("flightphase") == "GND").cast(pl.Float64).mean().alias("segment_gnd"),
            (pl.col("flightphase") == "CL").cast(pl.Float64).mean().alias("segment_cl"),
            (pl.col("flightphase") == "DE").cast(pl.Float64).mean().alias("segment_de"),
            (pl.col("flightphase") == "LVL").cast(pl.Float64).mean().alias("segment_lvl"),
            (pl.col("flightphase") == "CR").cast(pl.Float64).mean().alias("segment_cr"),
            (pl.col("flightphase") == "NA").cast(pl.Float64).mean().alias("segment_na"),
        ]

    # -------- apply aggregations (one row per segment) --------
    row = segment.select(aggs)

    # -------- dominant phase name (string) --------
    phase_cols   = ["segment_gnd", "segment_cl", "segment_de",
                    "segment_lvl", "segment_cr", "segment_na"]
    phase_labels = ["GND",         "CL",         "DE",
                    "LVL",         "CR",         "NA"]

    if all(c in row.columns for c in phase_cols):
        vals = row.select(phase_cols).row(0)  # tuple of 6 floats
        dominant_idx = max(range(len(vals)), key=lambda i: vals[i])
        dominant_name = phase_labels[dominant_idx]
        row = row.with_columns(
            pl.lit(dominant_name).alias("flightphase_dominant")
        )

    # -------- normalize: make all numeric columns Float64 for safe concat --------
    numeric_types = (
        pl.Int8, pl.Int16, pl.Int32, pl.Int64,
        pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
        pl.Float32, pl.Float64,
    )

    cols_to_cast = [
        c for c, dt in zip(row.columns, row.dtypes)
        if dt in numeric_types
    ]

    row = row.with_columns(
        [pl.col(c).cast(pl.Float64) for c in cols_to_cast]
    )

    return row





In [25]:
# Lit of Aircraft types which are not in OpenAPI
from openap import prop, FuelFlow
import numpy as np
import pandas as pd
from openap.phase import FlightPhase


def include_more(flight_trajectory):
    flights_info = pd.read_parquet(os.path.join("data", "flightlist_train.parquet"))
    aircraft_info = pd.read_csv(os.path.join("data", "ALL_DATA.csv"))

    flight_name = flight_trajectory["flight_id"][0]
    # ----- get icao row with typecode ------
    get_icao_row = aircraft_info.loc[aircraft_info["icao"] == flight_trajectory["typecode"][0]]

    # ------ add aircraft information ------
    flight_trajectory["wake"] = get_icao_row["wake"].values[0]
    flight_trajectory["engine_type"] = get_icao_row["engine_type"].values[0]
    flight_trajectory["engine_number"] = get_icao_row["engine_number"].values[0]

    # ------ get flight phase and add into df------
    # Compute time since first sample, in seconds
    ts0 = flight_trajectory["timestamp"][0]
    # Polars Datetime -> duration in milliseconds or nanoseconds depending on dtype
    # We'll convert to seconds as float
    ts = (flight_trajectory["timestamp"] - ts0).dt.total_seconds()

    # Extract numpy arrays for FlightPhase
    alt = flight_trajectory["altitude"].to_numpy()         # ft
    spd = flight_trajectory["groundspeed"].to_numpy()      # kts
    roc = flight_trajectory["vertical_rate"].to_numpy()    # ft/min
    tsec = ts.to_numpy()                    # seconds since start

    fp = FlightPhase()
    fp.set_trajectory(tsec, alt, spd, roc)

    labels = fp.phaselabel()
    flight_trajectory["flightphase"] = labels

    # now search takeoff-landing to get flight duration total and where the segment is starting
    flight_row = flights_info.loc[flights_info["flight_id"] == flight_name]
    flight_start = flight_row["takeoff"]
    flight_end = flight_row["landed"]
    flight_duration = (flight_end - flight_start).dt.seconds/60
    flight_duration = flight_duration.to_numpy()
    flight_trajectory["flight_duration"] = flight_duration[0]

    return flight_trajectory


In [28]:
# Do the training data
# Prepare training set
DO_FINAL = True                # run loop
segments_w_extra = False  # combine fuel with trajectory and extra rows
BUILD_TRAIN = True       # build final dataframe
save_combined = False    # write per-segment combined parquet?
data = "FINAL" # TRAIN for training data and RANK for sub data


# Load data - fuel_train includes fuel consumption in segments
DATA_ORIGIN = Path("data")
if data == "RANK":
    DATA = os.path.join(DATA_ORIGIN, "fuel_rank_submission.parquet")
elif data == "TRAIN":
    DATA = os.path.join(DATA_ORIGIN, "fuel_train.parquet")
elif data == "FINAL":
    DATA = os.path.join(DATA_ORIGIN, "fuel_final_submission.parquet")

df = pl.read_parquet(DATA)  # eager (fits your 5 GB fine)
# or lazy (more efficient for filtering/aggregation before loading)
ds = pl.scan_parquet("data/part-*.parquet")

BASE_CONTEXT = 5
MAX_CONTEXT  = 60
REQ_COLS = ["latitude","longitude","altitude"]

feature_parts: list[pl.DataFrame] = []
count_1 = 0
count_2 = 0
count_3 = 0
if DO_FINAL:
    for col in df.select(["flight_id","start","end","fuel_kg"]).iter_rows(named=True):
        fid = col["flight_id"]
        context = BASE_CONTEXT

        if segments_w_extra:
            segment = flight_search_extra(fid, col["start"], col["end"], context=context, data=data)
            while needs_more_context(segment) and context < MAX_CONTEXT:
                context += BASE_CONTEXT
                segment = flight_search_extra(fid, col["start"], col["end"], context=context)
        else:
            segment = flight_search(fid, col["start"], col["end"], data=data)
        count_1 += 1
        if segment.is_empty():
            continue

        # 1-row DF for flight-level fields
        flight_info = pl.DataFrame([col])
        count_2 += 1
        joined = dataframes_combine(flight_info, segment)
        if joined.is_empty():
            continue
        count_3 += 1
        # ---- add one-row feature summary for this segment ----
        if BUILD_TRAIN:
            row = build_final_row(joined)
            if not row.is_empty():
                feature_parts.append(row)

        # ---- (optional) persist the combined segment ----
        if save_combined:
            out_dir = (os.path.join(DATA_ORIGIN, "postprocessed", "fuel_flight_assigned_extra")
                       if segments_w_extra else
                       os.path.join(DATA_ORIGIN, "postprocessed", "fuel_flight_assigned"))
            os.makedirs(out_dir, exist_ok=True)
            joined.write_parquet(os.path.join(out_dir, f"{fid}.parquet"))  # add a unique suffix if needed

# ---- after loop: build & write final training DF ----
if BUILD_TRAIN and feature_parts:
    final_train = pl.concat(feature_parts, how="vertical", rechunk=True)
    out_train = os.path.join(DATA_ORIGIN, "postprocessed", "training", "FINAL_CLEAN_v2_1.parquet")
    os.makedirs(os.path.dirname(out_train), exist_ok=True)
    final_train.write_parquet(out_train)

